## GUI Tunning Tools for Computer Vision

In this notebook, we are going to use bqplots and ipywidget to create visual interractive tools that can be used to tune computer vision images and videos


**1. Importing packages for the GUI Tuner**

In [1]:
from ipywidgets.widgets import Label, FloatProgress, FloatSlider, Button, IntSlider
from ipywidgets.widgets import interact, interactive, fixed, interact_manual
from ipywidgets.widgets import Layout, HBox, VBox
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
import cv2
import numpy as np
import bqplot as bq
import time
import threading

**2. Tuner for threshold variation**

In [2]:
#Function for thresholding
def thresh_func(holda):
    #reading in an image
    image = mpimg.imread('car_images/solidWhiteRight.jpg')
    #image = mpimg.imread('car_images/image_1.jpg')
    #getting dimensions and making copy of the image
    ysize = image.shape[0]
    xsize = image.shape[1]
    treated = np.copy(image)

    #setting image threshold
    red_thresh = holda
    green_thresh = holda
    blue_thresh = holda
    thresh = [red_thresh, green_thresh, blue_thresh]

    # get pixels below threshold
    thresh_px = (image[:,:,0]<thresh[0])|(image[:,:,1]<thresh[1])|(image[:,:,2]<thresh[2])

    #set the threshold pixels as zero
    treated[thresh_px] = [0,0,0]

    #Show output
    plt.imshow(treated)

interact(thresh_func, holda = FloatSlider(value=0,
                                                  min = 0,
                                                max = 255,
                                                description = 'threshold',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')))

interactive(children=(FloatSlider(value=0.0, description='threshold', layout=Layout(margin='0 0 5px 0', width=…

<function __main__.thresh_func(holda)>

**3. Edge detection tuner**

This GUI Tuner is designed to tune parameters for edge detection

In [3]:
#Function for thresholding
def edge_func(kernel_size,low_thresh,high_thresh):
    #reading in an image
    image = mpimg.imread('car_images/solidWhiteRight.jpg')
    #image = mpimg.imread('car_images/image_1.jpg')
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #implement gaussian filter
    blur_gray = cv2.GaussianBlur(gray, (kernel_size,kernel_size), 0)
    #Applying Canny Edge detection
    edges = cv2.Canny(blur_gray, low_thresh, high_thresh)

    #show output
    plt.imshow(edges,cmap = 'Greys_r')

interact(edge_func, kernel_size = IntSlider(value=1,
                                                  min = 1,
                                                max = 31,
                                                step =2,
                                                description = 'kernel size',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
                                              low_thresh = FloatSlider(value=0,
                                                  min = 0,
                                                max = 255,
                                                description = 'low threshold',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')),\
                                              high_thresh = FloatSlider(value=0,
                                                  min = 0,
                                                max = 255,
                                                description = 'high threshold',
                                                layout =Layout(width = '490px', margin = '0 0 5px 0')))

interactive(children=(IntSlider(value=1, description='kernel size', layout=Layout(margin='0 0 5px 0', width='4…

<function __main__.edge_func(kernel_size, low_thresh, high_thresh)>

**3. Lane Finder tuner**

In [10]:
def lane_func(rho,threshold,min_len,max_len):
    #reading in an image
    # Read in and grayscale the image
    image = mpimg.imread('car_images/solidWhiteRight.jpg')
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 11
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

    # Define our parameters for Canny and apply
    low_threshold = 100
    high_threshold = 50
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    # Next we'll create a masked edges image using cv2.fillPoly()
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    print(imshape)
    vertices = np.array([[(0,imshape[0]),(475, 320),(475, 320), (imshape[1],imshape[0])]], dtype=np.int32)
    print(vertices)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    #rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    #threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    #min_line_length = 40 #minimum number of pixels making up a line
    #max_line_gap = 25   # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                                min_len, max_len)

    # Iterate over the output "lines" and draw lines on a blank image
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)

    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((edges, edges, edges)) 

    # Draw the lines on the edge image
    lines_edges = cv2.addWeighted(color_edges, 0.8, line_image, 1, 0) 
    plt.imshow(lines_edges)



interact(lane_func, rho = IntSlider(value=2,
                                      min = 1,
                                    max = 10,
                                    step =1,
                                    description = 'Rho',
                                    layout =Layout(width = '490px', margin = '0 0 5px 0')),\
                                  threshold = IntSlider(value=100,
                                      min = 1,
                                    max = 200,
                                    description = 'Threshold',
                                    layout =Layout(width = '490px', margin = '0 0 5px 0')),\
                                  min_len = IntSlider(value=0,
                                      min = 1,
                                    max = 100,
                                    description = 'Min lane length',
                                    layout =Layout(width = '490px', margin = '0 0 5px 0')),\
                                  max_len = IntSlider(value=20,
                                      min = 1,
                                    max = 100,
                                    description = 'Max lane length',
                                    layout = Layout(width = '490px', margin = '0 0 5px 0')))

interactive(children=(IntSlider(value=2, description='Rho', layout=Layout(margin='0 0 5px 0', width='490px'), …

<function __main__.lane_func(rho, threshold, min_len, max_len)>